In [4]:
import os
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") 


chat = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name='gpt-3.5-turbo',
    temperature=0.0
)


In [5]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)



In [6]:
from langchain.vectorstores import Pinecone as LPPinecone
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone

api_key = os.environ.get('PINECONE_API_KEY') 

# configure client
pc = Pinecone(api_key=api_key)

model_name = 'text-embedding-ada-002'
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") 

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=os.environ['OPENAI_API_KEY']
)

index_name = 'pilot2'
index = pc.Index(index_name)

text_field = "text"  # the metadata field that contains our text
vectorstore = LPPinecone(
    index, embed.embed_query, text_field
)



C:\Users\Fiberta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
C:\Users\Fiberta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.pinecone.Pinecone` was deprecated in langchain-community 0.0.18 and will be removed in 0.2.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
C:\Users\Fiberta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2

In [19]:
from openai import OpenAI
query = "what do you know about bill gates?"
openai_client = OpenAI(
  api_key = os.environ['OPENAI_API_KEY'],
)

res = openai_client.embeddings.create(
    input=[query],
    model=model_name
)
# retrieve from Pinecone
xq = res.data[0].embedding

# # get relevant contexts (including the questions)
res = index.query(vector=xq, top_k=2, include_metadata=True)

In [48]:
# res

In [10]:
limit = 3750

def retrieve(query):
    openai_client = OpenAI(
        api_key = os.environ['OPENAI_API_KEY'],
        )

    res = openai_client.embeddings.create(
        input=[query],
        model=model_name
    )

    # retrieve from Pinecone
    xq = res.data[0].embedding

    # get relevant contexts
    res = index.query(vector=xq, top_k=5, include_metadata=True)
    contexts = [
        x.metadata['text'] for x in res.matches
    ]

    # build our prompt with the retrieved contexts included
    prompt_start = (
        """ You are an AI assistant that helps people find information about Marketo.
            All questions will be about Marketo. Assistant should always be polite and helpful. 
            Always include references to documentation in assistant answers.\n\n"""+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    return prompt

In [11]:
query_with_contexts = retrieve(query)
query_with_contexts

" You are an AI assistant that helps people find information about Marketo.\n            All questions will be about Marketo. Assistant should always be polite and helpful. \n            Always include references to documentation in assistant answers.\n\nContext:\nInterviewed by Marketing Rockstar Guides!\nThanks a ton to Josh Hill of Marketing Rockstar Guides for his interview with me that came out today.\nAnybody reading this surely knows Josh, and probably knows me a little bit too. I'm not really rocking at his level, but then again, the rules have changed of late!\n\nQuestion: what do you know about bill gates?\nAnswer:"

In [18]:
def complete(prompt):
    res = openai_client.completions.create(
        prompt=prompt,
        temperature=0,
        max_tokens=400,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        model="gpt-3.5-turbo"
    )
    return res
complete(query_with_contexts)

NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}

In [2]:
complete(query_with_contexts)

NameError: name 'query_with_contexts' is not defined

Old code

In [22]:
# query = "what do you know about bill gates?"

# vectorstore.similarity_search(query, k=6)


In [23]:
# def augment_prompt(query: str):
#     results = vectorstore.similarity_search(query, k=6)
#     source_knowledge = "\n".join([x.page_content for x in results])
#     augmented_prompt = f"""Using the contexts below, answer the query.

#     Contexts:
#     {source_knowledge}

#     Query: {query}"""
#     return augmented_prompt

In [24]:
# messages = [
#     SystemMessage(content="""You are an AI assistant that helps people find information about Marketo.
#                   All questions will be about Marketo. Assistant should always be polite and helpful. 
#                  Always include references to documentation in assistant answers.""")
#     # HumanMessage(content="Hi AI, how are you today?")
#     # AIMessage(content="I'm great thank you. How can I help you?"),
#     # HumanMessage(content="I'd like to understand string theory.")
# ]

In [25]:

# # query = "what do you know about bill gates?"


# prompt = HumanMessage(
#     content=augment_prompt(
#         "Who is bill gates?"
#     )
# )
# res = chat(messages + [prompt])
# print(res.content)